# 1. Original model prediction

In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import pickle as pk
import numpy as np
import metrics as utils_metrics
import data_processing as dp
import models_old as models
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
out_path = Path("./result_path/Original_model_prediction")
out_path.mkdir(parents=True, exist_ok=True)
trainFileName='paper'
save_model_path="./Retraining_model"
predict_and_save(data_dict, model_name, save_model_path, trainFileName, out_path)

2024-12-24 19:52:28.353844: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


500/500 [==============================] - 2s 3ms/step
Saving done


In [3]:
from data_processing import encode_seq_array
from data_processing import encode_ep_class_neg
from tensorflow.keras.models import load_model
TCR_PAD_LENGTH = 20
EP_PAD_LENGTH = 15

def fix(data_ori):
    data=pd.read_csv(data_ori)
    #'CDR3.beta', 'antigen_epitope','mhc.a','label','negative.source','license'
    data.rename(columns={'CDR3B':'cdr3','Epitope':'antigen.epitope'},inplace=True)
    df=data[['cdr3', 'antigen.epitope','Affinity']]
    df.loc[df.Affinity==0, 'antigen.epitope']=''
    df=data[['cdr3', 'antigen.epitope']]    
    return df

def process_data(train_pos_data, testFilePath): 
    aa_vec = pk.load(open('atchley.pk', 'rb'))
    
    df_train = pd.read_csv(train_pos_data)
    
    df_pre=fix(testFilePath)
    train_ep_vc = df_train['antigen.epitope'].value_counts()
    
    data_dict = {'df_pre': df_pre,
                  'vc_train': train_ep_vc, #epitope list
                  #'vc_test': test_ep_vc, #epitope list
                }
    
    for set_type in ['pre']:
        label = 'df_' + set_type
        if label not in data_dict:
            continue
        data_dict['X_{}'.format(set_type)] = encode_seq_array(data_dict['df_{}'.format(set_type)]['cdr3'], aa_vec, True, TCR_PAD_LENGTH)
        data_dict['y_enc_{}'.format(set_type)] = encode_seq_array(data_dict['df_{}'.format(set_type)]['antigen.epitope'], aa_vec, True, EP_PAD_LENGTH)
    
        data_dict['y_cls_{}'.format(set_type)] = encode_ep_class_neg(data_dict['df_{}'.format(set_type)]['antigen.epitope'], data_dict['vc_train'])
    return data_dict

def identity_loss(y_true, y_pred):
    return y_pred
def model_predict(data_dict, save_model_path, batch_size=128):
    exp = load_model(save_model_path, custom_objects={'identity_loss': identity_loss})
    X_test = data_dict['X_pre']
    y_enc_test = data_dict['y_enc_pre']
    y_cls_test = data_dict['y_cls_pre']

    predictions = exp.predict(x=[X_test, y_enc_test, y_cls_test],batch_size=1)
    pred_prob=predictions
    
    # n_test = data_dict['X_pre'].shape[0]
    # idx_test = np.arange(n_test)
    # n_batches_test = n_test//batch_size
    # print(n_batches_test)

    # pred_prob=[]
    # for k in range(n_batches_test):
    #     start_test = k * batch_size
    #     end_test = (k+1) * batch_size
    #     batch_idx_test = idx_test[start_test:end_test]
    #     X_batch_test = X_test[batch_idx_test]
    #     y_enc_batch_test = y_enc_test[batch_idx_test]
    #     y_cls_batch_test = y_cls_test[batch_idx_test]
    #     val = exp.test_on_batch(x=[X_batch_test, y_enc_batch_test, y_cls_batch_test], y=y_cls_batch_test)
    #     # print(X_batch_test)
    #     predictions = exp.predict(x=[X_batch_test, y_enc_batch_test, y_cls_batch_test], batch_size=1)
    #     pred_prob.extend(predictions)
    #     # print("Predictions (probabilities):", predictions)
    return pred_prob

def Original_model_prediction(train_pos_data, testFilePath, modelfile_path, result_path, batch_size=128):

    data_dict=process_data(train_pos_data, testFilePath)
    pred_prob=model_predict(data_dict, modelfile_path, batch_size=128)
    
    data = pd.read_csv(testFilePath)
    probability = data[['Epitope', 'CDR3B', 'Affinity']]
    probability = probability.rename(columns={'Affinity': 'y_true'})
    probability['y_prob'] = pred_prob
    probability['y_pred'] = probability['y_prob'].apply(lambda x: 1 if x >= 0.5 else 0)
    probability.to_csv(f'{result_path}probability.csv', index=False)
    print("Saving done")

In [ ]:
testfile_path="../data/test.csv"
train_pos_data='positive.csv'
modelfile_path="../Original_model/MCMC.h5"
modelfile_path="./Retraining_model/model_MCMC_paper.h5"
result_path="../result_path/Original_model_prediction"
Original_model_prediction(train_pos_data,testfile_path,modelfile_path,result_path,batch_size=128)

# 2.Model retraining

In [6]:
from data_processing import encode_seq_array
from data_processing import encode_ep_class_neg
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

import pandas as pd
import pickle as pk
import numpy as np
import metrics as utils_metrics
import data_processing as dp
import model_1228 as models
from pathlib import Path
def fix(data_ori):
        data=pd.read_csv(data_ori)
        #'CDR3.beta', 'antigen_epitope','mhc.a','label','negative.source','license'
        data.rename(columns={'CDR3B':'cdr3','Epitope':'antigen.epitope'},inplace=True)
        df=data[['cdr3', 'antigen.epitope','Affinity']]
        df.loc[df.Affinity==0, 'antigen.epitope']=''
        df=data[['cdr3', 'antigen.epitope']]
        return df
def identity_loss(y_true, y_pred):
    return y_pred
def model_predict(data_dict, save_model_path, batch_size=128):
    exp = load_model(f'{save_model_path}.h5', custom_objects={'identity_loss': identity_loss})
    X_test = data_dict['X_test']
    y_enc_test = data_dict['y_enc_test']
    y_cls_test = data_dict['y_cls_test']
    predictions = exp.predict(x=[X_test, y_enc_test, y_cls_test],batch_size=1)
    pred_prob=predictions
    return pred_prob

def predict_and_save(data_dict, save_model_path,testfile_path, result_path):
    pred_prob=model_predict(data_dict, save_model_path, batch_size=128)
    data = pd.read_csv(testfile_path)
    probability = data[['Epitope', 'CDR3B', 'Affinity']]
    probability = probability.rename(columns={'Affinity': 'y_true'})
    probability['y_prob'] = pred_prob
    probability['y_pred'] = probability['y_prob'].apply(lambda x: 1 if x >= 0.5 else 0)
    probability.to_csv(f'{result_path}probability.csv', index=False)
    print("Saving done")

def Model_retraining(trainfile, testfile, save_model_path, resultfile_path):
    TCR_PAD_LENGTH = 20
    EP_PAD_LENGTH = 15
    aa_vec = pk.load(open('atchley.pk', 'rb'))
    df_train=fix(trainfile)
    df_test=fix(testfile)
    train_ep_vc = df_train['antigen.epitope'].value_counts()
    data_dict = {'df_train': df_train,
                  'df_test': df_test,
                  'vc_train': train_ep_vc,}
    for set_type in ['train', 'test']:
        label = 'df_' + set_type
        if label not in data_dict:
            continue
        data_dict['X_{}'.format(set_type)] = encode_seq_array(data_dict['df_{}'.format(set_type)]['cdr3'], aa_vec, True, TCR_PAD_LENGTH)
        data_dict['y_enc_{}'.format(set_type)] = encode_seq_array(data_dict['df_{}'.format(set_type)]['antigen.epitope'], aa_vec, True, EP_PAD_LENGTH)
        data_dict['y_cls_{}'.format(set_type)] = encode_ep_class_neg(data_dict['df_{}'.format(set_type)]['antigen.epitope'], data_dict['vc_train'])
    
    n_epoch=120
    batch_size=128
    network_dict = models.make_full_network_neg(None)
    train_output = models.train(network_dict, data_dict, n_epoch, save_model_path,batch_size)
    predict_and_save(data_dict, save_model_path,testfile, resultfile_path)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [ ]:
trainfile_path ="../data/train.csv"
testfile_path="../data/test.csv"
save_models_path="../Retraining_model/Retraining_model.pth"
result_path="../result_path/Retraining_model_prediction"
Model_retraining(trainfile_path,testfile_path,save_models_path,result_path) 

# 3. Retraining_model_prediction

In [15]:
from data_processing import encode_seq_array
from data_processing import encode_ep_class_neg
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import pickle as pk
import numpy as np
import metrics as utils_metrics
import data_processing as dp
import model_1228 as models
from pathlib import Path
def fix(data_ori):
        data=pd.read_csv(data_ori)
        #'CDR3.beta', 'antigen_epitope','mhc.a','label','negative.source','license'
        data.rename(columns={'CDR3B':'cdr3','Epitope':'antigen.epitope'},inplace=True)
        df=data[['cdr3', 'antigen.epitope','Affinity']]
        df.loc[df.Affinity==0, 'antigen.epitope']=''
        df=data[['cdr3', 'antigen.epitope']]
        return df
def identity_loss(y_true, y_pred):
    return y_pred
def model_predict(data_dict, save_model_path, batch_size=128):
    exp = load_model(save_model_path, custom_objects={'identity_loss': identity_loss})
    X_test = data_dict['X_test']
    y_enc_test = data_dict['y_enc_test']
    y_cls_test = data_dict['y_cls_test']
    predictions = exp.predict(x=[X_test, y_enc_test, y_cls_test],batch_size=1)
    pred_prob=predictions
    return pred_prob

def Retraining_model_prediction(trainfile, testfile, save_model_path, result_path):

    TCR_PAD_LENGTH = 20
    EP_PAD_LENGTH = 15
    aa_vec = pk.load(open('atchley.pk', 'rb'))
    df_train=fix(trainfile)
    df_test=fix(testfile)
    train_ep_vc = df_train['antigen.epitope'].value_counts()
    data_dict = {'df_test': df_test,
                 'vc_train': train_ep_vc,}
    for set_type in ['test']:
        label = 'df_' + set_type
        if label not in data_dict:
            continue
        data_dict['X_{}'.format(set_type)] = encode_seq_array(data_dict['df_{}'.format(set_type)]['cdr3'], aa_vec, True, TCR_PAD_LENGTH)
        data_dict['y_enc_{}'.format(set_type)] = encode_seq_array(data_dict['df_{}'.format(set_type)]['antigen.epitope'], aa_vec, True, EP_PAD_LENGTH)
        data_dict['y_cls_{}'.format(set_type)] = encode_ep_class_neg(data_dict['df_{}'.format(set_type)]['antigen.epitope'], data_dict['vc_train'])
        
    pred_prob=model_predict(data_dict, save_model_path, batch_size=128)
    data = pd.read_csv(testfile)
    probability = data[['Epitope', 'CDR3B', 'Affinity']]
    probability = probability.rename(columns={'Affinity': 'y_true'})
    probability['y_prob'] = pred_prob
    probability['y_pred'] = probability['y_prob'].apply(lambda x: 1 if x >= 0.5 else 0)
    probability.to_csv(f'{result_path}probability.csv', index=False)
    print("Saving done")



In [ ]:
trainfile_path="../data/train.csv"
testfile_path="../data/Validation.csv"
modelfile_path="../Retraining_model/Retraining_model.pth"
result_path="../result_path/Retraining_model_prediction"
Retraining_model_prediction(trainfile,testfile_path,modelfile_path,result_path)
